In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16

import numpy as np
import matplotlib.pyplot as plt

In [30]:

# 기존 이미지  갯수 늘리기:전처리
# train_data_gen = ImageDataGenerator(rescale=1./255)
# train_generator = train_data_gen.flow_from_directory('../fig/my_images/train', target_size=(150, 150), batch_size=5, class_mode='binary')

train_data_gen = ImageDataGenerator(rescale=1./255)
train_generator = train_data_gen.flow_from_directory('../fig/my_images/train', target_size=(150, 150), batch_size=5, class_mode='binary')

test_data_gen = ImageDataGenerator(rescale=1./255)  # 테스트 데이터는 정규화만
test_generator = test_data_gen.flow_from_directory('../fig/my_images/test', target_size=(150, 150), batch_size=5, class_mode='binary')

Found 895 images belonging to 2 classes.
Found 377 images belonging to 2 classes.


In [31]:
## transfer learning
transfer_vgg16 = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3))
transfer_vgg16.trainable = False
transfer_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [32]:
finetune_vgg16 = Sequential()
finetune_vgg16.add(transfer_vgg16)
finetune_vgg16.add(Flatten())
finetune_vgg16.add(Dense(64, activation='relu'))
finetune_vgg16.add(Dropout(0.3))
finetune_vgg16.add(Dense(1, activation='sigmoid'))
finetune_vgg16.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                524352    
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 15,239,105
Trainable params: 524,417
Non-trainable params: 14,714,688
_________________________________________________________________


In [33]:
finetune_vgg16.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['acc'])

checkpoint = ModelCheckpoint(filepath = './reco_family.hdf5', save_best_only=True)
erarly_stop = EarlyStopping(patience=10)

In [34]:
history = finetune_vgg16.fit(train_generator, epochs=100, batch_size = 20, validation_data=test_generator, callbacks=[checkpoint, erarly_stop])

Epoch 1/100
179/179 [==============================] - 107s 593ms/step - loss: 0.0210 - acc: 0.9922 - val_loss: 1.8489e-04 - val_acc: 1.0000
Epoch 2/100
179/179 [==============================] - 120s 672ms/step - loss: 3.6545e-04 - acc: 1.0000 - val_loss: 7.0327e-05 - val_acc: 1.0000
Epoch 3/100
179/179 [==============================] - 115s 646ms/step - loss: 7.7090e-04 - acc: 1.0000 - val_loss: 1.5403e-05 - val_acc: 1.0000
Epoch 4/100
179/179 [==============================] - 116s 652ms/step - loss: 0.0025 - acc: 0.9989 - val_loss: 3.7641e-04 - val_acc: 1.0000
Epoch 5/100
179/179 [==============================] - 112s 628ms/step - loss: 0.0011 - acc: 1.0000 - val_loss: 1.5264e-06 - val_acc: 1.0000
Epoch 6/100
179/179 [==============================] - 114s 640ms/step - loss: 1.8640e-04 - acc: 1.0000 - val_loss: 7.3285e-07 - val_acc: 1.0000
Epoch 7/100
179/179 [==============================] - 115s 645ms/step - loss: 5.0722e-05 - acc: 1.0000 - val_loss: 7.6747e-07 - val_acc: 1.00

179/179 [==============================] - 112s 629ms/step - loss: 3.9934e-06 - acc: 1.0000 - val_loss: 3.4124e-10 - val_acc: 1.0000
Epoch 58/100
179/179 [==============================] - 113s 630ms/step - loss: 5.5322e-05 - acc: 1.0000 - val_loss: 3.1642e-10 - val_acc: 1.0000
Epoch 59/100
179/179 [==============================] - 112s 626ms/step - loss: 1.5107e-05 - acc: 1.0000 - val_loss: 3.3188e-10 - val_acc: 1.0000
Epoch 60/100
179/179 [==============================] - 111s 620ms/step - loss: 1.2606e-06 - acc: 1.0000 - val_loss: 2.6763e-10 - val_acc: 1.0000
Epoch 61/100
179/179 [==============================] - 115s 645ms/step - loss: 2.6752e-06 - acc: 1.0000 - val_loss: 2.8267e-10 - val_acc: 1.0000
Epoch 62/100
179/179 [==============================] - 111s 620ms/step - loss: 1.5757e-05 - acc: 1.0000 - val_loss: 2.7401e-10 - val_acc: 1.0000
Epoch 63/100
179/179 [==============================] - 111s 618ms/step - loss: 6.6032e-06 - acc: 1.0000 - val_loss: 2.4094e-10 - val_acc

In [35]:
finetune_vgg16.save('./reco_family_1.hdf5')